In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# My libraries
import sys
sys.path.append('../')
import utils

## General parameters

In [ ]:
numberOfTimeStep = 14
folders = ["s1", "s2", "s3", "s4"]

norm = "robustNorm"

device = torch.device('cuda:0')
print("Seleccionando la segunda GPU:", torch.cuda.get_device_name(device))

## Train phase

In [ ]:
# Always the same values, do not touch. 
in_dim_GCN = 1
out_dim_GCN = 1

# number of max epochs 
n_epochs = 1000

# Early stopping configuration
early_stopping_patience = 20

# Hyperparameters to be optimized (change this values)
h_dropout = [0.15,0.3, 0.45]
h_learning_rate = [1e-4, 1e-3, 1e-2, 5e-2]
h_decay = [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
h_hid_lay = [4, 8, 16, 32]
h_layers = [1, 2, 3]
seed = [42, 76, 124, 163, 192, 205, 221, 245, 293]

fc_layer = [[56, out_dim_GCN]]

# Parameters to define type of GCNN and type of output.
typeGCN = "standard_gcnn"
K = [0]

params = {# Hyperparameters
         'h_layers':h_layers, 'n_epochs':n_epochs, 
          'h_dropout': h_dropout, 'h_learning_rate': h_learning_rate, 
          'h_decay':h_decay, 'h_hid_lay': h_hid_lay, 'K':K,
          'fc_layer': fc_layer,
          # seed to set initialization hyperparameters
          'seed': seed, 
          # Type of output GCN
          'typeGCN': typeGCN,
          # Dimensions of GCN (input/output)
          'in_dim_GCN': in_dim_GCN, 'out_dim_GCN': out_dim_GCN,
          # Patiente
          'early_stopping_patience':early_stopping_patience}

In [ ]:
way_to_build_graph = "dtw"
numberOfFeatures = 56

In [ ]:
best_result_by_split = {}
for carp in range(len(folders)):
    torch.cuda.empty_cache()
    # Load data
    X_train_vec, X_val_vec, X_test_vec, y_train, y_val, y_test = utils.load_data(norm, device, carp, SG=True)
    A = pd.read_csv("../../step2_graphRepresentation/"+way_to_build_graph+"/"+folders[carp]+"/graph_Xtr_th_0.975.csv")
    A = torch.tensor(np.array(A), dtype=torch.float32)
    
        
    print("X_train_vec:", X_train_vec.shape)
    print("X_val_vec:", X_val_vec.shape)
    print("X_test_vec:", X_test_vec.shape)
    print("y_train:", y_train.shape)
    print("y_val:", y_val.shape)
    print("y_test:", y_test.shape)

    print("===========> TRAIN-VAL PHASE ==================")
    bestHyperparameters = utils.train_val_phase(A, X_train_vec, X_val_vec, y_train, y_val, params, device)
    print("<========== END TRAIN-VAL PHASE ===============")
    best_result_by_split[folders[carp]] = bestHyperparameters    

utils.saveBestHyperparameters(best_result_by_split, "../hyperparameters/"+way_to_build_graph+"/GNN_th_0.975.json")

torch.cuda.empty_cache()

Early stopping at epoch 74
Early stopping at epoch 25
Early stopping at epoch 22
Early stopping at epoch 24
Early stopping at epoch 38
Early stopping at epoch 25
Early stopping at epoch 176
Early stopping at epoch 37
Early stopping at epoch 89
Early stopping at epoch 59
Early stopping at epoch 62
Early stopping at epoch 158
Early stopping at epoch 35
Early stopping at epoch 24
Early stopping at epoch 124
Early stopping at epoch 21
Early stopping at epoch 33
Early stopping at epoch 26
Early stopping at epoch 42
Early stopping at epoch 65
Early stopping at epoch 22
Early stopping at epoch 28
Early stopping at epoch 38
Early stopping at epoch 23
Early stopping at epoch 26
Early stopping at epoch 71
Early stopping at epoch 55
Early stopping at epoch 80
Early stopping at epoch 35
Early stopping at epoch 39
Early stopping at epoch 22
Early stopping at epoch 38
Early stopping at epoch 30
Early stopping at epoch 37
Early stopping at epoch 37
Early stopping at epoch 21
Early stopping at epoch 2

## Validation phase

In [ ]:
best_result_by_split = utils.loadBestHyperparameters("../hyperparameters/"+way_to_build_graph+"/GNN_th_0.975.json")

typeOfGraph = "ProdGraph"
path_A = "graph_Xtr_th_0.975.csv"
results, importance_nodes, fc_classifiers, gnn_models = utils.val_model(best_result_by_split, typeOfGraph, params, folders, \
                                                                        norm, device, path_A, way_to_build_graph, SG=True)

keys = list(results.keys())
for c in range(len(folders)):
    print("================= SPLIT " + str(folders[c]) + " ===================")
    print(keys[1] + ": " + str(np.round(results[keys[1]][c]*100,2)))
    print(keys[2] + ": " + str(np.round(results[keys[2]][c]*100,2)))
    print(keys[3] + ": " + str(np.round(results[keys[3]][c]*100,2)))
    
print()
str_result = ""
filtered_results = {
    key: results[key][1:]  
    for key in keys
}

formatted_results = {"Model": "./Results_GCNN"}

column_names = ["Accuracy", "ROC AUC", "Sensitivity", "Specificity"]

for i, key in enumerate(keys[:4]):  
    average = np.mean(filtered_results[key])
    std = np.std(filtered_results[key])
    formatted_results[column_names[i]] = f"{np.round(average * 100, 2)} ± {np.round(std * 100, 2)}"

df = pd.DataFrame([formatted_results])


output_path = "../../../Results_GCNN/results.csv"
df.to_csv(output_path, index=False)

print(f"Results saved to {output_path}")

torch.cuda.empty_cache()

In [ ]:
torch.save(gnn_models['s1'], './saved_models/s1.pth')
